# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Столбцы в таблице обозначают:

* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

В рамках проекта на предстоит решить несколько подзадач, такие как:
* открыть и изучить входные данные
* преодбработать данные, удалив пропуски, заменив типы данных, обрабоботать повторы и сделать ламметизацию
* провести категоризацию данных
* ответить на несколько вопросов, описанных выше
* написать общий вывод


1. [Открытие данных](#1)
2. [Предобработка данных](#2)
    * [Обработка пропущенных значения](#2.1)
    * [Замена типов данных](#2.2)
    * [Обработка дубликато](#2.3)
    * [Лемматизация](#2.4)
    * [Категоризация данных](#2.5)
    
3. [Ответы на вопросы](#3)
4. [Общий вывод](#4)

# Шаг 1. Откройте файл с данными и изучите общую информацию
<a id="1"></a>

In [207]:
import pandas as pd #импортируем библиотеку pandas в pd
from pymystem3 import Mystem #Одна из библиотек с функцией лемматизации на русском языке — pymystem3
m = Mystem()

In [208]:
import pandas as pd #импортируем библиотеку pandas в pd
data = pd.read_csv("/datasets/data.csv") #читаем файл 
data.info() #открываем информацию по файлу
display(data) #выведем на экран таблицу функцией display(), чтобы просмотреть значения в таблиц
data.tail() #последние 5 строк
data.sample() #случайная строка

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4060,0,339221.668471,52,среднее,1,Не женат / не замужем,4,F,пенсионер,0,118599.872418,получение образования


# Шаг 2. Предобработка данных 
<a id="2"></a>

### Обработка пропусков
<a id="2.1"></a>

In [209]:
data.isna().sum() # проверяем таблицу на пропуски

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

* Таким образом в двух столбцах, у нас пропущенные значения, поэтому заменим Nan на среднее значение выборки в столбце days_employed(потому что разброс данных велик)  и медиану(потому что для зарплаты целесообразнее использовать медиану) для total_income. 
* Проверку результата произведем, оторбразив уникальные значения столбца
* Уже на данном этапе становится понятно, что этот столбец не нужен будет в ходе решения, но для обработки пропусков необходимо это сделать.

In [210]:
data["days_employed"] = data["days_employed"].fillna(data["days_employed"].mean()) # заменим пропуски на среднее значение в данном столбце
data["days_employed"].value_counts() #выведем уникальные значения и сгруппируем 

 63046.497661    2174
-986.927316         1
-4236.274243        1
-1238.560080        1
-3047.519891        1
                 ... 
-2849.351119        1
-5619.328204        1
-448.829898         1
-1687.038672        1
-582.538413         1
Name: days_employed, Length: 19352, dtype: int64

In [211]:
data["total_income"] = data["total_income"].fillna(data["total_income"].median()) # заменим пропуски на медиану в столбце с зарплатой 
data["total_income"].value_counts() #выведем уникальные значения и сгруппируем

145017.937533    2175
112874.418757       1
104381.857170       1
182036.676828       1
122421.963500       1
                 ... 
133299.194693       1
115080.782380       1
84896.781597        1
153838.839212       1
150014.128510       1
Name: total_income, Length: 19351, dtype: int64

In [212]:
data.isna().sum() # проверяем таблицу на пропуски

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Замена типа данных
<a id="2.2"></a>

Приведем days_employed и total_income к типу int

In [213]:
data["days_employed"] =  data["days_employed"].astype(int) # перевод типа float к int методом astype(), который оптимален для этого действия
data['total_income'] = data['total_income'].astype(int) #приведем аналогичное действие к эжемесечной зарплате, так будет репрезентативнее
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [214]:
data["family_status_id"] =  data["family_status_id"].astype("int8") # перевод типа int64 к int8 методом astype(), который сэкономит память
data["education_id"] =  data["education_id"].astype("int8") #аналогично
data["education_id"] =  data["education_id"].astype("bool")
data["family_status_id"] =  data["family_status_id"].astype("bool")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null bool
family_status       21525 non-null object
family_status_id    21525 non-null bool
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: bool(2), int64(5), object(5)
memory usage: 1.7+ MB


Таким образом, мы days_employed приведен в целочисленный тип данных
Однако, по моему мнению, данный столбец несущественен в решении данной задачи, в связи с тем, что он заполнен ошибочными данными, которые не влияют на результаты других столбцов, его значения очень странные, стоило бы в реальной жизни запросить данные еще, вдруг произошла техническая ошибка, к тому же максимальное значение столбца, по модулю, не выглядит адекватным

In [215]:
days_employed_max = data["days_employed"].max()
days_employed_max_uear = days_employed_max/365
days_employed_max_uear

1100.6986301369864

1100 лет?????? Удалим данный столбец.

In [216]:
data = data.drop(["days_employed"],  axis=1) # применим метод dpop, в качестве аргумента название столбца, axis=1 - параметр удаление столбца)
data #выведем на экран

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,False,женат / замужем,False,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,True,женат / замужем,False,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,True,женат / замужем,False,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,True,женат / замужем,False,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,True,гражданский брак,True,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...
21520,1,43,среднее,True,гражданский брак,True,F,компаньон,0,224791,операции с жильем
21521,0,67,среднее,True,женат / замужем,False,F,пенсионер,0,155999,сделка с автомобилем
21522,1,38,среднее,True,гражданский брак,True,M,сотрудник,1,89672,недвижимость
21523,3,38,среднее,True,женат / замужем,False,M,сотрудник,1,244093,на покупку своего автомобиля


Таблица выше, показывает, что изменение типа произошло успешно, как и удаление столбца days_employed

### Обработка дубликатов
<a id="2.3"></a>

**Вывод**

In [217]:
data_duplicated = data[data.duplicated()] #для наглядности выведем DataFrame с дубликатами,делаем вывод что явные дубликаты присутсвуют
data_duplicated

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,41,среднее,True,женат / замужем,False,F,сотрудник,0,145017,покупка жилья для семьи
4182,1,34,ВЫСШЕЕ,False,гражданский брак,True,F,сотрудник,0,145017,свадьба
4851,0,60,среднее,True,гражданский брак,True,F,пенсионер,0,145017,свадьба
5557,0,58,среднее,True,гражданский брак,True,F,пенсионер,0,145017,сыграть свадьбу
7808,0,57,среднее,True,гражданский брак,True,F,пенсионер,0,145017,на проведение свадьбы
8583,0,58,высшее,False,Не женат / не замужем,True,F,пенсионер,0,145017,дополнительное образование
9238,2,34,среднее,True,женат / замужем,False,F,сотрудник,0,145017,покупка жилья для сдачи
9528,0,66,среднее,True,вдовец / вдова,True,F,пенсионер,0,145017,операции со своей недвижимостью
9627,0,56,среднее,True,женат / замужем,False,F,пенсионер,0,145017,операции со своей недвижимостью
10462,0,62,среднее,True,женат / замужем,False,F,пенсионер,0,145017,покупка коммерческой недвижимости


Можно убедиться в этом же, применим метод duplicated  и sum()

In [218]:
data.duplicated().sum() # количество строк с явными дубликатами

54

Удалим явные дубликаты, методом drop_duplicates(),перезапишем индексы и сделаем проверку.

In [219]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()


0

После удаления явных дубликатов перейдем к ручному поиску дубликатов с учетом регистра.

In [220]:
data.columns #для удобства 

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')

In [221]:
index = ['education','family_status','gender','income_type','purpose'] # cоздадим список
for i in index: 
    data[i] = data[i].str.lower() #присвоим значение одного столбца к столбцу с нижним регистром
data["education"].value_counts()# для проверки выведем уникальные значения столбцов

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [222]:
data["family_status"].value_counts()

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [223]:
data["gender"].value_counts() #проверим и пол

f      14189
m       7281
xna        1
Name: gender, dtype: int64

Как мы видим, что здесь появилось интересное значение XNA, пожалуй удалим его его, так как в решении задачи расчетов для гендера нет, скорее всего это ошибочное значение. После этого имеет смысл проверить остальные столбцы, на всякий случай, вдруг обнаружим что-нибудь интересное.

In [224]:
data = data[data['gender'] != 'xna']
data["gender"].value_counts()

f    14189
m     7281
Name: gender, dtype: int64

In [225]:
data["children"].value_counts()

 0     14106
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Так, странно но у одной семьи 20 детей, а у одной -1, скорее всего в первом случае лишний 0, а в другом минус, подкоректируем эти значения.

In [226]:
data["children"] = data["children"].replace(-1,1)

In [227]:
data["children"].value_counts() #проверили, -1 отсутсвует

0     14106
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [228]:
children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median() #найдем медиану для всех строк в столбце, где значение не равно 20.
data['children'] = data['children'].replace(20, children_median) #метод replace идеально подходит в этом случае

In [229]:
data["children"].value_counts()

0    14182
1     4856
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [230]:
data.duplicated().sum() #после изменения регистра нашлось еще несколько строчек дубликатов

17

In [231]:
data = data.drop_duplicates().reset_index(drop=True) #аналогично примеру выше
data.duplicated().sum() #дубликатов нет

0

Итак, стоит отметить, что данный путь можно было сократить, изначально применив нижний регистр по всем значениям и удалить уже "явные" дубликаты, но так как это учебный проект, решил сделать таким образом, вспомнив все основные методы.
Данные дубликаты в первом случае - техническая ошибка, во втором - это человеческий фактор:)

In [232]:
data.shape



(21453, 11)

### Лемматизация
<a id="2.4"></a>

In [233]:
data["purpose_lemma"] = data["purpose"].apply(m.lemmatize) #Создадим столбец с леммой
from collections import Counter #для подсчета упоминания в тексте
text = [] #cоздадим лист с леммами
for i in data["purpose_lemma"]: #пройдемся циклом по нему
    text.extend(i) #в данном случае нужно использовать метод extend, который в отличие от append, добавляет все элеметты списка, расширяя его
Counter(text) # контейнер для нахождения числа упоминания в столбце

Counter({'покупка': 5896,
         ' ': 33569,
         'жилье': 4460,
         '\n': 21453,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6350,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 478,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'подержанный': 486,
         'получение': 1314,
         'высокий': 1374,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607})

Выше - это частота, упоминаемых в столбце слов, предлагаю создать список с категориями, в дальнейшем применить стеминг для того, чтобы категоризировать данные

In [234]:
from nltk.stem import SnowballStemmer #импортируем библиотеку для стемминга
russian_stemmer = SnowballStemmer('russian') # передаем функция для стеминга рускоязычного текста
text = ["жилье", "свадьба", "автомобиль", "образование", "недвижимость"] # создаем список лемм
kategorii = [] #создаем пустой список категорий
for word in text: #проходим по циклу
    kategorii.append(russian_stemmer.stem(word)) #добавляем в список результаты стемминга по каждому слову 
print(kategorii) #выводим на экран стемминг слов

['жил', 'свадьб', 'автомобил', 'образован', 'недвижим']


**Вывод** 
Таким образом, мы произвели лематизацию и нашли стемминг слов, чтобы создать в следующем пункте, категории для целей получения кредита.

### Категоризация данных
<a id="2.5"></a>

In [235]:
def purposes_categories(purposes_kredit): #создадим функцию, которая будет проверять есть ли значение леммы в стемминге
    if ("жил")  in  purposes_kredit or ("недвижим") in purposes_kredit: #логичекое выражение, потому что и в тов и в том случае будет категория недвижимость
        return "недвижимость"
    elif "свадьб" in purposes_kredit: #аналоично
        return "свадьба"
    elif "автомобил" in purposes_kredit:
        return "автомобиль"
    elif "образован" in purposes_kredit:
        return "образование"
data["purposes_categories"] = data["purpose"].apply(purposes_categories) #для применения функции к столбцу применим метод apply()

def children_categories(children): # аналогично выше
    if children == 0:
        return "бездетная_семья"
    elif  0< children < 3:
        return "малодетная_семья"
    elif children >=3:
        return "многодетная_семья"
data['children_categories'] = data["children"].apply(children_categories)

def total_income_categories(total_income): # аналогично выше
    if 50000 > total_income:
        return "меньше 50"
    elif 50000 <= total_income < 100000:
        return "от 50 до 100"
    elif 100000 <= total_income < 150000:
        return "от 100 до 150"
    elif 150000 <= total_income: 
        return "от 150"  
data['total_income_categories'] = data["total_income"].apply(total_income_categories)
data['total_income_categories'].value_counts()

от 150           9183
от 100 до 150    7807
от 50 до 100     4091
меньше 50         372
Name: total_income_categories, dtype: int64

**Вывод**

Таким образом, мы разделили цели взятия кредита на 4 категории, которые наиболее часто встречаюся в списке из лемм, жилье и недвижемость объеденили в одну категорию.
С детьми, я посчитал, что целесообразнее выделить три категории(общепринятые) : бездетные, малодетные и многодетные семьи.
По заработной плане, исходя из статей в интернете, было принято решение сделать срезы по 50 тысяч.

# Шаг 3. Ответьте на вопросы 
<a id="3"></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [236]:
data_pivot = data.pivot_table(index = ["children_categories"], values = ["debt"]) #создадим сводную таблицу, 
data_pivot

,debt
children_categories,
бездетная_семья,0.075604
малодетная_семья,0.092515
многодетная_семья,0.081579


**Вывод**

Как мы видим по сводной таблице, бездетные семьи показывают меньше склонны к просрочкам по кредитам - это связано с тем, что в семьях с детьми часть дохода уходит на содержание детей. Стоит отметить, что многодетные семьи в отличии от малодетных более отвественно относятся к кредитам, это логично, у таких семей более высокая степень ответсвенности во многих аспектах жизни, что конечно же влияет на платежеспособность. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [237]:
data_pivot = data.pivot_table(index = ["family_status"], values = ["debt"]) #создадим сводную таблицу, 
data_pivot

,debt
family_status,
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093494
женат / замужем,0.075452
не женат / не замужем,0.097509


**Вывод**

Интересно, но люди в разводе или не в браке, а так же овдовевшие платят чаще в срок, нежели в браке или гражданском браке. Возможно, это признак того, что люди более скорпулезно относятся со своими деньгами и нет отвественности за кого-либо. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [238]:
data_pivot = data.pivot_table(index = ["total_income_categories"], values = ["debt"]) #создадим сводную таблицу, 
data_pivot

,debt
total_income_categories,
меньше 50,0.061828
от 100 до 150,0.084668
от 150,0.079059
от 50 до 100,0.080909


**Вывод**
Люди с доходом меньше 50 тысяч платят стабильнее, остальных групп. Влияние дохода в других группах есть, но это небольшое.

- Как разные цели кредита влияют на его возврат в срок?

In [239]:
data_pivot = data.pivot_table(index = ["purposes_categories"], values = ["debt"]) #создадим сводную таблицу, 
data_pivot

,debt
purposes_categories,
автомобиль,0.093590
недвижимость,0.072340
образование,0.092200
свадьба,0.080034


**Вывод**

Как мы видим, среди 4 наших категорий, чаще всего платят в срок с целью кредита недвижимость, чаще просрочка у тех кому нужен кредит на автомобиль и образование. Кредиты на свадьбу просрачивают реже - примерно как и в случае недвижимости.

# Шаг 4. Общий вывод
<a id="4"></a>

Итак, исходя из исследования ответим на несколько вопросов:
1) зависимость между наличием детей и возвратом в срок:
- бездетные семьи имеют 7,5% просроченных платажей
- малодетные семьи 8,1% просроченных платажей
- многодетные семьи 9,2%

Таким образом, наличие детей уже отражается на платежеспособность в срок от 0.6% до 1.7%. Стоит отметить, что многодетные семьи платят в срок чаще, нежели малодетные семьи, думаю это завивисит от ответсвенности, в многодетных семьях родители распределяют бюджет намного лучше, так как их зона ответственности повышается и к кредиту они могут подходить наиболее обдуманнее.

2) связь семейного положения и возвратом в срок:
- для вдовцы/вдовца 6.6%
- для людей в разводе 7.1%
- для женатых людей 7.5%
- для гражданского брака 9.3%
- для тех, кто не имеет отношений 9.7%

Таким образом, присутсвует связь, интересно для тех, кто был в семейных отношений платят в срок чаще, в отличии от других групп. Стоит обратить внимание при одобрении кредита, что люди не в отношениях чаще чем другие имеют задолжность по кредитам.

3) связь уровня дохода и возвратом в срок:
- меньше 50 -  6.1%
- от 50 до 100 -  8%
- от 100 до 150 - 8.4%
- от 150 - 8%

Исходя из выше данных, люди с небольшим доходов платят стабильнее других групп. Если заработок выше 50 тысяч рублей, значение повышается, но не критично.

4) связь цели кредита и возвратом в срок:
- для недвижимости - 7.2%
- для свадьбы - 8%
- для образования - 9.2%
- для автомобиля - 9.3%

Исходя из этих результатов, кредиты для свадьбы и недвижимости платят в срок чаще, нежели другие. Образование и автомобиль чаще не платят в срок.


In [240]:
#!c1.4
